In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
from datetime import datetime, timedelta
import pandas as pd

In [2]:
titlesize = 16
labelsize = 16
legendsize = 16
xticksize = 20
yticksize = xticksize

plt.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
plt.rcParams['legend.handletextpad'] = 0.5
plt.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
plt.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
plt.rcParams['font.size'] = 12
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['axes.labelsize'] = labelsize
plt.rcParams['axes.titlesize'] = titlesize
plt.rcParams['figure.figsize'] = (10, 6)

plt.rc('xtick', labelsize=xticksize)
plt.rc('ytick', labelsize=yticksize)
plt.rc('legend', fontsize=legendsize)

In [3]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Задание 1. Отличия от эксперимента
Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv и experiment_users.csv.

В качестве ответа введите p-value, округлённое до 3-го знака после точки.

In [4]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_users = read_database('experiment_users.csv')

In [5]:
# количество пользователей в группах
df_users['pilot'].value_counts()

0    11769
1    11564
Name: pilot, dtype: int64

In [6]:
# считаем значения метрики
begin_date = datetime(2022, 3, 16)
end_date = datetime(2022, 3, 23)
df_metrics = (
    df_sales
    [(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]
    .groupby('user_id')[['price']].sum()
    .reset_index()  
)

In [7]:
df = pd.merge(df_users, df_metrics, on='user_id', how='left').fillna(0)

In [8]:
df.head()

,user_id,pilot,price
0,0ffc65,0,0.0
1,b962b9,0,0.0
2,7ea63f,0,0.0
3,7f9a61,0,0.0
4,459e55,0,2160.0


In [9]:
data_control = df[df['pilot'] == 0]['price']
data_pilot = df[df['pilot'] == 1]['price']
stats.ttest_ind(data_control, data_pilot)

Ttest_indResult(statistic=-1.2837567415000515, pvalue=0.19923983306424942)

# Задание 2. Среднее время между покупками

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv.

В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [34]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [35]:
# для каждого пользователя считаем количество покупок, дату первой и последней покупки
df = df_sales.groupby('user_id')[['date']].agg(['count', 'min', 'max'])

In [38]:
df.columns = [x[1] for x in df.columns]

In [40]:
df = df[df['count'] >= 2]

In [41]:
# количество секунд между первой и последней покупкой
df['delta'] = (df['max'] - df['min']).dt.total_seconds()

C:\Users\KULIK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
# суммарное время между покупками
sum_delta = df['delta'].sum()

In [43]:
# суммарное количество периодов между покупками
count_periods = df['count'].sum() - len(df)

In [44]:
# среднее = суммарное время / количество периодов
answer = sum_delta / count_periods / 3600 / 24
print('answer:', int(round(answer)))

answer: 17


In [3]:
!pip install pydantic

  Found existing installation: typing-extensions 4.4.0
    Uninstalling typing-extensions-4.4.0:
      Successfully uninstalled typing-extensions-4.4.0


In [4]:
from pydantic import BaseModel

In [5]:
class User(BaseModel):
    login: str
    subscribed: bool = False
    age: int

In [6]:
user = User(login='winner2003', age='22', city='Moscow')

# Платформа A/B тестирования состоит из 4 блоков
- DataService - доступ к сырым данным
- MetricService - вычисление метрик для экспериментов
- ExperimentService - Проверка статистической значимости
- SplittingServce

# Задание 3. Реализация метода для проверки статистической значимости
Напишите метод get_pvalue класса ExperimentsService. Описание метода есть в шаблоне решения ниже.

Скопируйте код шаблона в py-файл, допишите недостающие части, сохраните изменения и отправьте файл в проверяющую систему.

Для самопроверки внизу шаблона есть пример теста решения. При проверке решения будут использоваться другие тестовые данные.

In [10]:
import numpy as np
from pydantic import BaseModel
from scipy import stats


class Design(BaseModel):
    """Дата-класс с описание параметров эксперимента."""
    statistical_test: str


class ExperimentsService:

    def get_pvalue(self, metrics_a_group, metrics_b_group, design):
        """Применяет статтест, возвращает pvalue.

        :param metrics_a_group (np.array): массив значений метрик группы A
        :param metrics_b_group (np.array): массив значений метрик группы B
        :param design (Design): объект с данными, описывающий параметры эксперимента
        :return (float): значение p-value
        """
        if design.statistical_test == 'ttest':
            # YOUR_CODE_HERE
            _, pvalue = stats.ttest_ind(metrics_a_group, metrics_b_group)
            return pvalue
        else:
            raise ValueError('Неверный design.statistical_test')

In [11]:
metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
design = Design(statistical_test='ttest')
ideal_pvalue = 0.612219

experiments_service = ExperimentsService()
pvalue = experiments_service.get_pvalue(metrics_a_group, metrics_b_group, design)
np.testing.assert_almost_equal(ideal_pvalue, pvalue, decimal=4)
print('simple test passed')

simple test passed
